In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

from xgboost import XGBClassifier
import xgboost as xgb


from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score


In [2]:
%%time
train_df = pd.read_csv(r"C:\Users\TimeUser\Desktop\contest\train.csv")
test_df = pd.read_csv(r"C:\Users\TimeUser\Desktop\contest\test.csv")

Wall time: 1min 2s


In [3]:
n_splits = 9
skf = StratifiedKFold(n_splits,True,615)
feats = ['var_'+str(i) for i in range(200)]

In [4]:
params = {
    'objective':'binary:logistic',
    #'booster':'gbdt',
    'eval_metric':['auc'],
    'eta':0.02,
    #'num_leaf':35,
    'subsample':0.8,
    'colsample_bytree':0.8,
    #'colsample_bynode':0.8
    #'colsample_bylevel':0.8
    #'sub_feature':0.05,
    'reg_lambda':100,
    'max_depth':6,
    'min_child_weight':50,
    'silent':1
    #'min_gain_to_split':0.2
}

In [5]:
tst_xgb = xgb.DMatrix(test_df[feats])

In [6]:
%%time
oof_pred = np.zeros(train_df.shape[0])
tst_pred = np.zeros(test_df.shape[0])
imp = np.zeros(len(feats))
rounds = 0
for trn_ids,val_ids in skf.split(train_df.index,train_df['target']):
    rounds += 1
    print('Round %d'%rounds)
    trn_xgb = xgb.DMatrix(train_df.iloc[trn_ids,:][feats],train_df.iloc[trn_ids,:]['target'])
    val_xgb = xgb.DMatrix(train_df.iloc[val_ids,:][feats],train_df.iloc[val_ids,:]['target'])
    watchlist = [(trn_xgb, 'train'), (val_xgb, 'valid')]
    evals_result = {}
    model = xgb.train(params,trn_xgb, num_boost_round = 1000000,evals = watchlist,early_stopping_rounds=500,evals_result=evals_result,verbose_eval =500)
    
    oof_pred[val_ids] = model.predict(val_xgb)
    
    tst_pred += model.predict(tst_xgb)/n_splits
    
    #imp += model.feature_importance('gain')/n_splits
print(roc_auc_score(train_df['target'].values,oof_pred))

Round 1
[0]	train-auc:0.571198	valid-auc:0.573422
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 500 rounds.
[500]	train-auc:0.91492	valid-auc:0.858797
[1000]	train-auc:0.943503	valid-auc:0.879736
[1500]	train-auc:0.955932	valid-auc:0.887202
[2000]	train-auc:0.963206	valid-auc:0.890857
[2500]	train-auc:0.968296	valid-auc:0.89266
[3000]	train-auc:0.972343	valid-auc:0.893687
[3500]	train-auc:0.976146	valid-auc:0.894245
[4000]	train-auc:0.979667	valid-auc:0.894593
[4500]	train-auc:0.983097	valid-auc:0.894511
Stopping. Best iteration:
[4073]	train-auc:0.980159	valid-auc:0.894632

Round 2
[0]	train-auc:0.584671	valid-auc:0.578343
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 500 rounds.
[500]	train-auc:0.913412	valid-auc:0.858233
[1000]	train-auc:0.942611	valid-auc:0.880664
[1500]	train-auc:0.955437	valid-auc:0.888852
[200

In [7]:
sub_df = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df["target"] = tst_pred
sub_df.to_csv("xgb_sumb_0.8952AUC.csv", index=False)